In [118]:
import queue
import re
from urllib.parse import urlparse
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import csv
import time
import os
from reppy.robots import Robots

In [13]:
def is_absolute(url):
    """Determine whether URL is absolute."""
    return bool(urlparse(url).netloc)

In [14]:
options = webdriver.ChromeOptions()
options.add_argument("headless")
#driver = webdriver.Chrome(chrome_options=options)
driver = webdriver.Chrome(executable_path='F:\Stevens_Masters\Spring 19\Web Mining\Week 3\chromedriver_win32\chromedriver.exe')


In [108]:
email_addresses = []
q = queue.Queue()
q.put("https://www.stevens.edu/")
mydata=[["URL","DIRECTORY"]]
myFile = open("linkinfo.csv", 'w',newline='')
with myFile:
    writer = csv.writer(myFile)
    writer.writerows(mydata)

In [109]:
if not os.path.exists("root"):
    os.mkdir("root")
    

In [110]:
fileDir = os.path.dirname(os.path.realpath('__file__'))

os.chdir('C://Users//DEEPUAISHU//root')


In [123]:
url_list=[]
for i in range(300000):
    #print("round {}.format",i)
    url = q.get()
    #print(url)
    if(url not in url_list):
        url_list.append(url)
        #print(url)
        #finding crawl delay and reducing 4 seconds to it(only for stevens.edu)
        robots = Robots.fetch('http://stevens.edu/robots.txt')
        crawl_delay=robots.agent('foo').delay
        time.sleep(crawl_delay-4)
         
        driver.get(url)
        page_name="root"
        #print(driver.page_source)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        o=urlparse(url)
        #saving source into a text file
        path=s=o.path
        if(path=="/" or path=="/#" ):
            path=page_name
        else:
            path=page_name+path

        #print(path)
        
        page_data=[soup.contents]
        way=path
        #replacing unwanted characters while creating file name
        p=way.replace("/","_")
        p=p.replace("<","")
        p=p.replace(">","")
        p=p.replace("?","")
        p=p.replace(":","")
        p=p.replace("*","")
        p=p.replace("\t"," ")
        
        with open(p+".txt",'w+',encoding="utf-8") as f:
            for e in page_data:
                f.write(str(e))
                f.close
        #saving link and path to a csv file    
        mydata=[[url,path]]
        myFile = open("linkinfo.csv", 'a',newline='',encoding="utf-8")
        with myFile:
            writer = csv.writer(myFile)
            writer.writerows(mydata)
            #finding email address in a page(professor program logic)
        email_addresses += re.findall("\S+@stevens.edu", soup.get_text())
        email_addresses = list(set(email_addresses))
        links = soup.find_all('a')
        for link in links:
            u = link.get('href')
            if not is_absolute(u):
                u = urljoin(url, u)
                #print(u)
            if "stevens.edu" in u:
                q.put(u)
    else:
        continue

In [122]:
with open("email.txt", "w+") as f:
    for e in email_addresses:
        f.write(e + "\n")